# LIMS probabilistic linkage

In [ ]:
import easylinkage

In [ ]:
# Imports

import os
import pandas as pd
import numpy as np
from easylinkage import preprocessing

# Settings

encoding = 'ISO-8859-1'

# Variables

INPUTDATA = "../data/LIMS/probabilisticLinkage/"

TARGET_FILENAME = "New lims extract.xlsx"
QUERY_FILENAME = "LINC Gold Standard - Original Testing Registers"

# Reading functions

def read_targetdata():
    df = pd.read_excel(os.path.join(INPUTDATA, TARGET_FILENAME), sheet = "Sheet 1")
    df = df[["PATID", "patnumber", "accessnumber", "FIRSTNAME", "NAME", "SEX", "birthdate", "LOCNAME"]]
    df = df.rename(dict(zip(df.columns, ["patid1", "patid2", "patid3", "firstname", "surname", "sex", "birthdate", "locname"])), axis = 1)
    df = df.drop_duplicates()
    # First Name
    df.firstname = preprocessing.clean(df.firstname)
    # Surname
    df.surname = preprocessing.clean(df.surname)
    # Sex
    df.loc[df.sex == 1, ["sex"]] = "m"
    df.loc[df.sex == 2, ["sex"]] = "f"
    df.loc[df.sex == 0, ["sex"]] = np.nan
    # Date
    df.birthdate = pd.to_datetime(df["birthdate"], errors = "coerce")
    # Locname
    df.locname = preprocessing.clean(df.locname)
    df.locname = df.locname.str.replace('\d+', '')
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

def read_sourcedata():
    return

df = read_targetdata()
dft = df[50:200].copy()
dft = dft.reset_index(drop=True)
dfq = df[:100].copy()
dfq = dfq.reset_index(drop=True)


## Indexing

In [ ]:
from easylinkage import Index

indexer = Index()
indexer.softblock(left_on = ["firstname", "surname"], right_on = ["firstname", "surname"])
pairs = indexer.index(dfq, dft)

## Compare

In [ ]:
from easylinkage import Compare

comp = Compare()
comp.name('firstname', 'firstname')
comp.name('surname', 'surname')
comp.birthdate("birthdate", "birthdate")
comp.sex("sex", "sex")
comp.location("locname", "locname")

dfc = comp.compute(pairs, dfq, dft)

In [1]:
from easylinkage import ECMClassifier

ecm = ECMClassifier()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
from easylinkage import ECMClassifier

In [ ]:
class EnsembleECMClassifier(ECMClassifier):
     
    def __init__(self, ensemble_size = 10, init = "jaro", max_iter = 100, binarize = None, atol = 1e-4, use_col_names = True):
        ECMClassifier.__init__(self, init = init, max_iter = max_iter, binarize = binarize, atol = atol, use_col_names = use_col_names)
        self.ensemble_size = ensemble_size
        self.clfs = []
        self.fit_columns = []
        
    def _sample_features(self, df):
        def _get_feature_dict(df):
            features = collections.defaultdict(list)
            for c in df.columns:
                features[c.split("---")[0]] += [c]
            return features
        features = _get_feature_dict(df)
        ensemble = []
        for _ in range(0, self.ensemble_size):
            feats = []
            for k, v in features.items():
                feats += [random.choice(v)]
            ensemble += [feats]
        return ensemble
            
    def fit(self, comparison_vectors):
        dfc = comparison_vectors
        ensemble = self._sample_features(dfc)
        for cols in ensemble:
            clf = ECMClassifier()
            clf.fit(dfc[cols])
            self.clfs += [clf]
            self.fit_columns += [clf.raw_fit_columns]
            
    def fit_predict(self, comparison_vectors):
        dfc = comparison_vectors
        self.fit(dfc)
        return self.predict(dfc)
            
    def predict(self, comparison_vectors):
        dfc = comparison_vectors
        prd = []
        for cols, clf in zip(self.fit_columns, self.clfs):
            prd += [clf.predict(dfc[cols])]
        prd = pd.concat(prd, axis = 1)
        prd = prd.mode(axis = 1)
        return probas
         
    def prob(self, comparison_vectors):
        dfc = comparison_vectors
        prd = []
        for cols, clf in zip(self.fit_columns, self.clfs):
            prd += [clf.prob(dfc[cols])]
        prd = pd.concat(prd, axis = 1)
        prd = prd.median(axis = 1)
        return prd
    
ecm = EnsembleECMClassifier()
ecm.fit(dfc)
prob = ecm.prob(dfc)

In [2]:
import recordlinkage

In [3]:
recordlinkage.__version__

'0.13.2+6.g2524fad'

In [ ]:
uu = BaseECMClassifier(binarize = 0.5)

In [ ]:
ecm = BaseECMClassifier(init = "random", binarize = 0.5)
ecm.fit(uu)

In [ ]:
uu

## Feature ensembles

In [ ]:
def get_feature_dict(df):
    features = collections.defaultdict(list)
    for c in df.columns:
        features[c.split("_")[0]] += [c]
    return features

features = get_feature_dict(dfc)

In [ ]:
B = 10

def sample_features(df, B):
    features = get_feature_dict(df)
    ensemble = []
    for _ in range(0, B):
        feats = []
        for k, v in features.items():
            feats += [random.choice(v)]
        ensemble += [feats]
    return ensemble

ensemble = sample_features(dfc, 10)

In [ ]:
?SimpleImputer

In [ ]:
## Probabilistic linkage

ecm = classifiers.ECMClassifier(binarize = 0.5, max_iter = 1000, atol = 1e-10)
ecm.fit(dfc)

In [ ]:
ecm.fit(comparison_vectors=)

In [ ]:
min_proba = 0.0

def raw_match(dfc):
    sums = dfc.sum(axis=1)
    return sums / dfc.shape[1]

def ecm_probabilities(dfc):
    ecm = classifiers.ECMClassifier(binarize = 0.5, max_iter = 1000, atol = 1e-10)
    ecm.fit(dfc)
    probas = ecm.prob(dfc)
    return probas

def ensemble_ecm_probabilities(dfc, ensemble):
    probas = []
    for cols in ensemble:
        probas += [ecm_probabilities(dfc[cols])]
    probas = pd.concat(probas, axis = 1)
    probas = probas.median(axis = 1)
    return probas

def linkage(dfc, dfq, dft, ensemble):
    raws  = raw_match(dfc)
    probas = ensemble_ecm_probabilities(dfc, ensemble)
    dfl = pd.DataFrame(data = {"index.q": probas.index.labels[0],
                               "index.t": probas.index.labels[1],
                               "raw": raws.values,
                               "proba": probas.values,
                               "firstname.q": dfq.iloc[probas.index.labels[0]]["firstname"].values,
                               "surname.q": dfq.iloc[probas.index.labels[0]]["surname"].values,
                               "birthdate.q": dfq.iloc[probas.index.labels[0]]["birthdate"].values,
                               "firstname.t": dft.iloc[probas.index.labels[1]]["firstname"].values,
                               "surname.t": dft.iloc[probas.index.labels[1]]["surname"].values,
                               "birthdate.t": dft.iloc[probas.index.labels[1]]["birthdate"].values})
    dfl = dfl[["index.q", "index.t", "raw", "proba", "firstname.q", "surname.q", "birthdate.q", "firstname.t", "surname.t", "birthdate.t"]]
    dfl = dfl[dfl.proba >= min_prob]
    dfl = dfl.sort_values(by = ["index.q", "raw"], ascending = [True, False])
    dfl = dfl.reset_index(drop = True)
    return dfl

dfl = linkage(dfc, dfq, dft, ensemble)

In [ ]:
dfl.sort_values(by = "proba", ascending = False)

In [ ]:
probas = pd.concat(probas, axis = 1)

In [ ]:
probas.median(axis = 1)

In [ ]:
p = 0.8

import itertools
from collections import defaultdict
from scipy import spatial

def get_lowest(df, cols):
    low = 0
    f1 = 0
    f2 = 0
    for k in itertools.combinations(cols, 2):
        cos = spatial.distance.cosine(df[k[0]].values, df[k[1]].values)
        if cos > low:
            low = cos
            f1 = k[0]
            f2 = k[1]
    return f1, f2

def generate_pool(df, p):
    features = defaultdict(list)
    features_names = set([x.split("_")[0] for x in df.columns])
    for feature in sorted(features_names):
        vf = [col for col in df.columns if feature in col]
        if len(vf) == 1:
            features[feature] = vf
            continue
        f1, f2 = get_lowest(df, vf)
        V = [f1, f2]
        features[feature].append(f1)
        features[feature].append(f2)
        vf.remove(f1)
        vf.remove(f2)
        while vf:
            f = vf.pop()
            c = 0
            for item in V:
                num = spatial.distance.cosine(df[f].values, df[item].values)
                if num > p and f not in features:
                    c += 1;
            if c == len(V):
                V.append(f)
                features[feature].append(f)
    return features

def convert_to_features_schema(features):
    features = list(dict(features).values())
    features_schema = list(itertools.product(*features))
    return features_schema

def get_features_schema(df):
    dfv = feature_vectors.reset_index(drop = True)
    features = generate_pool(dfv, p)
    features_schema = convert_to_features_schema(features)
    return features_schema

features_schema = get_features_schema(feature_vectors)

In [ ]:
def calculate_weights(df, features, Xm, Xu, w):
    ls = []
    djs = []
    for index, f in enumerate(features):
        match = (df.loc[Xm][f] - 1).sum(axis = 0)
        notmatch = (df.loc[Xu][f] - 0).sum(axis = 0)
        dj = match + notmatch
        djs.append(dj)
        if dj == 0:
            ls.append(index)
    if len(ls) > 0:
        w = np.zeros(w.shape)
        w[ls] = 1 / len(ls)
    else :
        s = sum(1 / d for d in djs)
        djs = [round(1 / (dj*s), 5) for dj in djs]
        w = np.asarray(djs).reshape(w.shape)
    return w

def feature_weights(df, Mm, Mu, e, w):
    Xm=set()
    Xu=set()
    tm, tu = 0, 0
    while(len(Xm)<Mm):
        t=np.dot(abs(df[~df.index.isin(Xm)].values-1),w)
        Xm.update(df[~df.index.isin(Xm)][t<=tm].head(Mm-len(Xm)).index) ### fill the seed until we reach Mm without repeating
        tm+=0.05
    while(len(Xu)<Mu):
        t=np.dot(abs(df[~df.index.isin(Xu)].values-0),w)
        inde=set(df[(~df.index.isin(Xu))][t<=tu].head(Mu-len(Xu)).index) ### make sure that no matching point is selected for not matching point
        Xu.update(inde-Xm)
        tu+=0.05
    wnew=calculate_weights(df,df.columns,Xm,Xu,w)
    while( np.array(abs(wnew-w)>e).any()):
        Xm=set()
        Xu=set()
        tm,tu=0,0
        w=wnew
        while(len(Xm)<Mm):
            t=np.dot(abs(df[~df.index.isin(Xm)].values-1),w)
            Xm.update(df[~df.index.isin(Xm)][t<=tm].head(Mm-len(Xm)).index)
            tm+=0.05
        while(len(Xu)<Mu):
            t=np.dot(abs(df[~df.index.isin(Xu)].values-0),w)
            inde=set(df[(~df.index.isin(Xu))][t<=tu].head(Mu-len(Xu)).index)
            Xu.update(inde-Xm)
            tu+=0.05
        wnew=calculate_weights(df,df.columns,Xm,Xu,w)
    return Xm,Xu

## Probabilities

In [ ]:
dfl[dfl.proba > 0.5].sort_values(by = "proba")

In [ ]:
clf = classifiers.ECMClassifier()

In [ ]:
clf = rl.ECMClassifier(binarize=0.5, max_iter=100, atol = 1e-4)
clf.fit(feature_vectors)

In [ ]:
feature_vectors

In [ ]:
sums = feature_vectors.sum(axis = 1)

In [ ]:
clf = rl.ECMClassifier()

In [ ]:
clf.fit(feature_vectors)

In [ ]:
?rl.ECMClassifier

In [ ]:
p = 0.8

import itertools
from collections import defaultdict
from scipy import spatial

def get_lowest(df, cols):
    low = 0
    f1 = 0
    f2 = 0
    for k in itertools.combinations(cols, 2):
        cos = spatial.distance.cosine(df[k[0]].values, df[k[1]].values)
        if cos > low:
            low = cos
            f1 = k[0]
            f2 = k[1]
    return f1, f2

def generate_pool(df, p):
    features = defaultdict(list)
    features_names = set([x.split("_")[0] for x in df.columns])
    for feature in sorted(features_names):
        vf = [col for col in df.columns if feature in col]
        if len(vf) == 1:
            features[feature] = vf
            continue
        f1, f2 = get_lowest(df, vf)
        V = [f1, f2]
        features[feature].append(f1)
        features[feature].append(f2)
        vf.remove(f1)
        vf.remove(f2)
        while vf:
            f = vf.pop()
            c = 0
            for item in V:
                num = spatial.distance.cosine(df[f].values, df[item].values)
                if num > p and f not in features:
                    c += 1;
            if c == len(V):
                V.append(f)
                features[feature].append(f)
    return features

def convert_to_features_schema(features):
    features = list(dict(features).values())
    features_schema = list(itertools.product(*features))
    return features_schema

def get_features_schema(df):
    dfv = feature_vectors.reset_index(drop = True)
    features = generate_pool(dfv, p)
    features_schema = convert_to_features_schema(features)
    return features_schema

In [ ]:
dfv = feature_vectors.reset_index(drop = True)
features = generate_pool(dfv, p)
features_schema = convert_to_features_schema(features)

In [ ]:
def calculate_weights(df, features, Xm, Xu, w):
    ls = []
    djs = []
    for index, f in enumerate(features):
        match = (df.loc[Xm][f] - 1).sum(axis = 0)
        notmatch = (df.loc[Xu][f] - 0).sum(axis = 0)
        dj = match + notmatch
        djs.append(dj)
        if dj == 0:
            ls.append(index)
    if len(ls) > 0:
        w = np.zeros(w.shape)
        w[ls] = 1 / len(ls)
    else :
        s = sum(1 / d for d in djs)
        djs = [round(1 / (dj*s), 5) for dj in djs]
        w = np.asarray(djs).reshape(w.shape)
    return w

def automatic_seed_selection(df, Mm, Mu, e, w):
    Xm=set()
    Xu=set()
    tm,tu=0,0
    while(len(Xm)<Mm):
        t=np.dot(abs(df[~df.index.isin(Xm)].values-1),w)
        Xm.update(df[~df.index.isin(Xm)][t<=tm].head(Mm-len(Xm)).index) ### fill the seed until we reach Mm without repeating
        tm+=0.05
    while(len(Xu)<Mu):
        t=np.dot(abs(df[~df.index.isin(Xu)].values-0),w)
        inde=set(df[(~df.index.isin(Xu))][t<=tu].head(Mu-len(Xu)).index) ### make sure that no matching point is selected for not matching point
        Xu.update(inde-Xm)
        tu+=0.05
    wnew=calculate_weights(df,df.columns,Xm,Xu,w)
    while( np.array(abs(wnew-w)>e).any()):
        Xm=set()
        Xu=set()
        tm,tu=0,0
        w=wnew
        while(len(Xm)<Mm):
            t=np.dot(abs(df[~df.index.isin(Xm)].values-1),w)
            Xm.update(df[~df.index.isin(Xm)][t<=tm].head(Mm-len(Xm)).index)
            tm+=0.05
        while(len(Xu)<Mu):
            t=np.dot(abs(df[~df.index.isin(Xu)].values-0),w)
            inde=set(df[(~df.index.isin(Xu))][t<=tu].head(Mu-len(Xu)).index)
            Xu.update(inde-Xm)
            tu+=0.05
        wnew=calculate_weights(df,df.columns,Xm,Xu,w)
    return Xm,Xu

In [ ]:
Mm = 200
Mu = 8000
e  = 0.5

def calculate_Xm_Xu(df, features):
    Xm = []
    Xu = []
    for k, v in features.items():
        z = len(v)
        w = np.full((z, 1), 1. / z)
        x1, x2 = automatic_seed_selection(df[v], Mm, Mu, e, w)
        Xm.append(x1)
        Xu.append(x2)    
    return Xm, Xu

In [ ]:
Xm, Xu = calculate_Xm_Xu(dfv, features)

In [ ]:
ALL = set().union(*Xm)

def calculate_Q(set0, set1):
    S00 = set0.intersection(set1)
    S11 = ALL - (set0.union(set1))
    S01 = set0 - set1
    S10 = set0 - set1
    Q = float((len(S00)*len(S11)) - (len(S01) * len(S10))) / ((len(S00)*len(S11)) + (len(S01) * len(S10)))
    return Q

In [ ]:
Qs = []
for f in itertools.combinations(range(0,len(Xm)),2):
    Qs.append((calculate_Q(Xm[f[0]],Xm[f[1]]),f[0],f[1]))

In [ ]:
from frameworks.SelfLearning import *
from sklearn.linear_model import LogisticRegression

models = []
X_features = []
for i, schema in enumerate(features_schema):
    model = SelfLearningModel(LogisticRegression(solver = "lbfgs"))
    models.append(model)
    X_features += [list(schema)]

In [ ]:
%%time
for i, model in enumerate(models):
    X = dfv[X_features[i]].values
    name = 'proba_' + str(i)
    dfv[name] = -1
    dfv.loc[Xm[i], name] = 1
    dfv.loc[Xu[i], name] = 0
    y = dfv[name].values
    model.fit(X, y)
    #dfv.loc[dfv[name] == -1, name] = model.predict_proba(dfv[dfv[name] == -1][X_features[i]].values)

In [ ]:
dfv

In [ ]:
dfv

In [ ]:
features_schema

In [ ]:
from recordlinkage import *

In [ ]:
from recordlinkage.standardise import *